# Merged CSV Files

In [ ]:
import pandas as pd
import glob


In [7]:
df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

youtube_spam_df = pd.concat(df_list, ignore_index=True)

youtube_spam_df.head()


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [6]:
files = glob.glob("Youtube*.csv")
files


['Youtube01-Psy.csv',
 'Youtube02-KatyPerry.csv',
 'Youtube03-LMFAO.csv',
 'Youtube04-Eminem.csv',
 'Youtube05-Shakira.csv']

In [8]:
youtube_spam_df.to_csv("youtube_spam.csv", index=False)
print("✅ youtube_spam.csv created successfully")


✅ youtube_spam.csv created successfully


In [9]:
youtube_spam_df.shape
youtube_spam_df["CLASS"].value_counts()


CLASS
1    1005
0     951
Name: count, dtype: int64

# Model Training (TF-IDF + Logistic Regression)

In [10]:
import pandas as pd

df = pd.read_csv("youtube_spam.csv")
df.head()


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [11]:
df.shape


(1956, 5)

In [12]:
df["CLASS"].value_counts()


CLASS
1    1005
0     951
Name: count, dtype: int64

In [13]:
#Text Cleaning

import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", " url ", text)
    text = re.sub(r"@\w+", " user ", text)
    text = re.sub(r"#\w+", " hashtag ", text)
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["CONTENT"].apply(clean_text)
df[["CONTENT", "clean_text"]].head()


,CONTENT,clean_text
0,"Huh, anyway check out this you[tube] channel: ...",huh anyway check out this you tube channel kob...
1,Hey guys check out my new channel and our firs...,hey guys check out my new channel and our firs...
2,just for test I have to say murdev.com,just for test i have to say murdev com
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,me shaking my sexy ass on my channel enjoy
4,watch?v=vtaRGgvGtWQ Check this out .﻿,watch v vtarggvgtwq check this out


In [14]:
#Train–Test Split

from sklearn.model_selection import train_test_split

X = df["clean_text"]
y = df["CLASS"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [15]:
#TF-IDF + Logistic Regression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2))),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)
print("✅ Model trained successfully")


✅ Model trained successfully


In [16]:
#Model Evaluation

from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9336734693877551
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       191
           1       0.94      0.94      0.94       201

    accuracy                           0.93       392
   macro avg       0.93      0.93      0.93       392
weighted avg       0.93      0.93      0.93       392



In [19]:
#Test on Instagram-like comments

sample_comments = [
    "Nice picture bro 🔥",
    "Click my link to earn money fast",
    "DM for paid promotion",
    "Love this post ❤️",
    "Win free iPhone now!!!",
    "Follow me and get free followers"
]

clean_samples = [clean_text(c) for c in sample_comments]

preds = model.predict(clean_samples)
probs = model.predict_proba(clean_samples)[:, 1]

for c, p, pr in zip(sample_comments, preds, probs):
    label = "SPAM 🚫" if p == 1 else "LEGIT ✅"
    print(f"{label} | {pr:.2f} | {c}")


LEGIT ✅ | 0.25 | Nice picture bro 🔥
SPAM 🚫 | 0.71 | Click my link to earn money fast
SPAM 🚫 | 0.59 | DM for paid promotion
LEGIT ✅ | 0.18 | Love this post ❤️
SPAM 🚫 | 0.55 | Win free iPhone now!!!
SPAM 🚫 | 0.69 | Follow me and get free followers


In [18]:
import joblib

joblib.dump(model, "instagram_spam_model.pkl")
print("✅ Model saved as instagram_spam_model.pkl")


✅ Model saved as instagram_spam_model.pkl


In this project, I developed an AI-based Instagram Spam Detection system to identify spam and legitimate comments on social media posts. Since a labeled Instagram spam dataset is not publicly available, I used the YouTube Spam Collection Dataset for training the model, as spam patterns are similar across social media platforms.

I performed text preprocessing including cleaning, normalization, and removal of noise such as URLs and special characters. I then applied TF-IDF vectorization with N-grams to assign weightage to commonly occurring spam words. Using these features, I trained a Logistic Regression classifier to classify comments as spam or non-spam.

The trained model achieved high accuracy (90%+) and was tested on Instagram-like comments to demonstrate its effectiveness. This project highlights the use of Natural Language Processing (NLP) and machine learning techniques to solve real-world problems related to social media security and content moderation.